In [1]:
!pip install scikit-learn --quiet


[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
from sklearn.cluster import KMeans

from pathlib import Path
import pickle

In [3]:
def get_financial_info(tickers):
    data = {}
    for ticker in tickers:
        info = yf.Ticker(ticker).info
        data[ticker] = {
            'dividendYield': info.get('dividendYield', None),
            'payoutRatio': info.get('payoutRatio', None),
            'profitMargins': info.get('profitMargins', None),
            'returnOnAssets': info.get('returnOnAssets', None),
            'returnOnEquity': info.get('returnOnEquity', None),
            'debtToEquity': info.get('debtToEquity', None),
            'revenueGrowth': info.get('revenueGrowth', None)
        }
    return pd.DataFrame.from_dict(data, orient='index')

In [4]:
from sklearn.preprocessing import StandardScaler

def preprocess_data(data):
    # Selecionar indicadores financeiros relevantes
    indicators = ['dividendYield', 'payoutRatio', 'profitMargins', 'returnOnAssets', 'returnOnEquity', 'debtToEquity', 'revenueGrowth']

    # Filtrar os dados apenas com os indicadores selecionados
    df = data[indicators].copy()  # Cópia do DataFrame para evitar chained assignment

    # Imputação estratégica dos valores ausentes
    df['dividendYield'] = df['dividendYield'].fillna(0)  # Preenche com 0 onde dividendYield é ausente
    df['payoutRatio'] = df['payoutRatio'].fillna(df['payoutRatio'].mean())  # Preenche com a média de payoutRatio onde ausente
    df['profitMargins'] = df['profitMargins'].fillna(df['profitMargins'].mean())  # Preenche com a média de profitMargins onde ausente
    df['returnOnAssets'] = df['returnOnAssets'].fillna(df['returnOnAssets'].mean())  # Preenche com a média de returnOnAssets onde ausente
    df['returnOnEquity'] = df['returnOnEquity'].fillna(df['returnOnEquity'].mean())  # Preenche com a média de returnOnEquity onde ausente
    df['debtToEquity'] = df['debtToEquity'].fillna(df['debtToEquity'].mean())  # Preenche com a média de debtToEquity onde ausente
    df['revenueGrowth'] = df['revenueGrowth'].fillna(df['revenueGrowth'].mean())  # Preenche com a média de revenueGrowth onde ausente

    # Normalizar os dados para evitar viés de escala
    scaler = StandardScaler()
    df_normalized = scaler.fit_transform(df)

    return df_normalized


In [5]:
def apply_clustering(data, max_clusters=10):
    distortions = []
    for i in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=i, random_state=42)
        kmeans.fit(data)
        distortions.append(kmeans.inertia_)

    # Escolher o número de clusters com base no método do cotovelo (elbow method)
    elbow_index = np.argmin(np.gradient(distortions)) + 1
    num_clusters = elbow_index if elbow_index > 1 else 2  # Garantir pelo menos 2 clusters

    # Aplicar K-Means com o número ideal de clusters
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    labels = kmeans.fit_predict(data)

    return kmeans, labels

In [6]:
def get_dividend_clusters(assets):
    # Obtendo os dados financeiros
    financial_data = get_financial_info(assets)

    # Pré-processando os dados
    processed_data = preprocess_data(financial_data)

    # Aplicando o algoritmo de agrupamento
    kmeans, labels = apply_clustering(processed_data)

    # Criando um dicionário para armazenar os tickers de cada cluster
    cluster_dict = {i: [] for i in range(kmeans.n_clusters)}

    # Preenchendo o dicionário com os tickers de cada cluster
    for i, label in enumerate(labels):
        cluster_dict[label].append(assets[i])

    return cluster_dict

# Testing

In [7]:
assets = [
    'VAMO3.SA',
    'VBBR3.SA',
    'VIVA3.SA',
    'VIVT3.SA',
    'VLID3.SA',
    'VULC3.SA',
    'WEGE3.SA',
    'WIZC3.SA',
    'YDUQ3.SA',
    'ZAMP3.SA'
]

clusters = get_dividend_clusters(assets)
clusters

{0: ['VAMO3.SA', 'VIVT3.SA', 'VULC3.SA', 'YDUQ3.SA', 'ZAMP3.SA'],
 1: ['VBBR3.SA', 'VIVA3.SA', 'VLID3.SA', 'WEGE3.SA', 'WIZC3.SA']}

In [8]:
path = Path("data/dividend_clusters.pkl")
path.parent.mkdir(exist_ok=True)

with open(path, mode="wb") as file:
    pickle.dump(clusters, file)

# Continuação

A partir dos clusters de empresas com comportamentos semelhantes de dividendos, será feito uma análise em cada clusters para identificar quais são as características mais relevantes (com mais correlação) para predição de dividendos.

[dividend_correlation_analysis.ipynb](dividend_correlation_analysis.ipynb)